# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

J. Liu  ->  J. Liu  |  ['J. Liu']
J. Li  ->  J. Li  |  ['J. Li']
J. Liu  ->  J. Liu  |  ['J. Liu']
Y. Wang  ->  Y. Wang  |  ['Y. Wang']
J. Li  ->  J. Li  |  ['J. Li']
H. Beuther  ->  H. Beuther  |  ['H. Beuther']
Arxiv has 75 new papers today
          5 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/5 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2505.14047


extracting tarball to tmp_2505.14047...

 done.
Retrieving document from  https://arxiv.org/e-print/2505.14073


extracting tarball to tmp_2505.14073...

...

 done.
Retrieving document from  https://arxiv.org/e-print/2505.14439


extracting tarball to tmp_2505.14439...

 done.
Retrieving document from  https://arxiv.org/e-print/2505.14686


extracting tarball to tmp_2505.14686...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2505.14686-b31b1b.svg)](https://arxiv.org/abs/2505.14686) | **HDO ice detected toward an isolated low-mass protostar with JWST**  |
|| K. Slavicinska, et al. -- incl., <mark>H. Beuther</mark> |
|*Appeared on*| *2025-05-21*|
|*Comments*| *Accepted for publication in ApJL. 10 pages, 4 figures, 1 table (+14 pages, 10 figures, 2 tables in appendix)*|
|**Abstract**|            Water is detected in environments representing every stage of star and solar system formation, but its chemical evolution throughout these stages remains poorly constrained. Deuterium ratios offer a means of probing chemical links between water in different cosmic regions because of their sensitivity to physicochemical conditions. Here, we present the first detection of the 4.1 $\mu$m HDO ice feature with JWST toward L1527 IRS, an isolated low-mass protostar that may eventually grow to a sun-like mass. We measure an ice HDO/H$_{2}$O ratio of 4.4$^{+3.7}_{-1.7}$$\times$10$^{-3}$, where the reported error is dominated by uncertainties in continuum definition and ice band strengths. This fraction is similar to the gas HDO/H$_{2}$O ratios measured in the warm ($>$100 K) inner cores of other low-mass protostellar envelopes and protoplanetary disks found in comparably isolated star-forming regions. Such a similarity tentatively supports the assumption that water vapor detected in these regions is not significantly altered by gas-phase reactions following ice sublimation. It also supports the hypothesis that pre- and protostellar water ice is largely inherited in a chemically unaltered state by outer protoplanetary disks. However, the fraction is a factor of $\sim$4-10 times higher than the gas HDO/H$_{2}$O ratios measured toward comets and low-mass protostars in clustered star-forming regions. This difference may be due to either gas-phase water reprocessing in protostellar envelopes and protoplanetary disks, or differences between prestellar conditions of isolated dense cores and the clustered star-forming regions that are more analogous to the environment in which our Sun formed.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2505.14047-b31b1b.svg)](https://arxiv.org/abs/2505.14047) | **The JCMT BISTRO-3 Survey: Variation of magnetic field orientations on parsec and sub-parsec scales in the massive star-forming region G28.34+0.06**  |
|| J. Hwang, et al. -- incl., <mark>J. Liu</mark> |
|*Appeared on*| *2025-05-21*|
|*Comments*| **|
|**Abstract**|            Magnetic fields play a significant role in star-forming processes on core to clump scales. We investigate magnetic field orientations and strengths in the massive star-forming clump P2 within the filamentary infrared dark cloud G28.34+0.06 using dust polarization observations made using SCUBA-2/POL-2 on the James Clerk Maxwell Telescope as part of the B-field In STar-forming Region Observations (BISTRO) survey. We compare the magnetic field orientations at the clump scale of ~2 parsecs from these JCMT observations with those at the core scale of ~0.2 parsecs from archival ALMA data, finding that the magnetic field orientations on these two different scales are perpendicular to one another. We estimate the distribution of magnetic field strengths, which range from 50 to 430 {\mu}G over the clump. The region forming the core shows the highest magnetic field strength. We also obtain the distribution of mass-to-flux ratios across the clump. In the region surrounding the core, the mass-to-flux ratio is larger than 1, which indicates the magnetic field strength is insufficient to support the region against gravitational collapse. Therefore, the change in the magnetic field orientation from clump to core scales may be the result of gravitational collapse, with the field being pulled inward along with the flow of material under gravity.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2505.14073-b31b1b.svg)](https://arxiv.org/abs/2505.14073) | **Zangetsu: A Candidate of Isolated, Quiescent, and Backsplash Ultra-Diffuse Galaxy in the COSMOS Field**  |
|| L. Wei, et al. -- incl., <mark>J. Li</mark> |
|*Appeared on*| *2025-05-21*|
|*Comments*| *25 pages, 11 figures*|
|**Abstract**|            Deep imaging surveys have changed our view of the low surface brightness (LSB) Universe. The "renaissance" of the low surface brightness dwarf galaxy population, as the prime example of such recent development, continues to challenge our understanding of galaxy formation. Here, We report the serendipitous discovery of Zangetsu, an isolated, quiescent, and distorted ultra-diffuse galaxy (UDG) candidate in the COSMOS field, using images from the Hyper Suprime-Cam Subaru Strategic Program (HSC-SSP). Zangetsu exhibits an extremely low central surface brightness ($\mathrm{\mu_{0,g}}=26.60\pm0.01$ mag arcsec$^{-2}$), a very shallow inner surface brightness profile ($\mathrm{n}_{\rm Sersic}=0.40\pm0.01$), and a large angular size ($\mathrm{R_e}\approx 10.44$ arcsec). Surprisingly, Zangetsu also has a quiescent stellar population ($\mathrm{g-i}=0.96$), an unusually elongated shape ($\mathrm{b/a}\sim 0.25$), and mild morphological asymmetry, making it a rare case among known UDGs. Surface brightness fluctuation analysis of HSC and Hubble Space Telescope (HST) images only provides a distance lower limit of $D>25.4$ Mpc (thus $\mathrm{R_e}>1.38$ kpc). However, Zangetsu remains an extreme outlier in the luminosity-size relation of known LSB galaxies, suggesting that it could be an exceptionally large and/or diffuse system. Classic internal or external UDG formation mechanisms alone struggle to explain such a system. A backsplash origin may account for its isolation and quiescent nature. This finding also raises the possibility that current works may overlook similarly extreme, elongated systems that could further our understanding of the LSB Universe.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2505.14384-b31b1b.svg)](https://arxiv.org/abs/2505.14384) | **Improving the Γ-functions Method for Vortex Identification**  |
|| Q. Xie, <mark>J. Liu</mark>, R. Erdélyi, <mark>Y. Wang</mark> |
|*Appeared on*| *2025-05-21*|
|*Comments*| *14 pages, 7 figures, accepted by Astronomy and Astrophysics*|
|**Abstract**|            Vortices have been observed at various heights within the solar atmosphere and are suggested to potentially play great roles in heating the solar upper atmosphere. Multiple automated vortex detection methods have been developed and applied to detect this http URL aim to improve the $\Gamma$-functions method for vortex identification by optimizing the value of $\Gamma_{1min}$ and the approach to calculate $\Gamma_1$ and $\Gamma_2$ used to determine vortex center and edge. In this way, we can detect vortices more accurately and enable more statistical studies that can improve our knowledge of the generation and evolution of vortices in the solar atmosphere. We apply the automated swirl detection algorithm (ASDA, one representative of $\Gamma$-functions method) with different parameters to various synthetic data, with each containing 1000 Lamb-Oseen vortices, and search for the optimal $\Gamma_{1min}$ and kernel size when calculating $\Gamma_1$ and $\Gamma_2$. We also compare another detection method using simulation and observational data to validate the results obtained from the synthetic data. The best performance is found with the Optimized ASDA, which combines different kernel sizes (5, 7, 9, and 11) to calculate $\Gamma_1$ and $\Gamma_2$ with a fixed $\Gamma_{1min}$ = 0.63 to detect vortex center. We find that more vortices can be detected by the Optimized ASDA with higher location, radius, and rotation speed accuracies. The above results are further confirmed by comparing vortices detected by the Optimized ASDA and the SWIRL method on CO5BOLD numerical simulation data and SST observational data.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2505.14439-b31b1b.svg)](https://arxiv.org/abs/2505.14439) | **Five New Sirius-Like White Dwarf + Main Sequence Star Systems with MagAO-X**  |
|| L. A. Pearce, et al. -- incl., <mark>J. Li</mark> |
|*Appeared on*| *2025-05-21*|
|*Comments*| *26 pages, 15 figures, 3 tables. Accepted to AJ. Data available at this https URL*|
|**Abstract**|            Most known white dwarfs in multiple systems with main sequence stars have been discovered with M-type companions, because the white dwarf causes detectable UV excess and bluer colors than expected from a single M star. Surveys have shown that the number of white dwarfs in Sirius-like systems within 100 pc of the Sun is lower than expected, suggesting that white dwarfs are being missed in the glare of their main sequence companions. In this work we have leveraged the angular resolution and high-contrast capabilities, as well as optimization for visible wavelengths, of the extreme adaptive optics instrument MagAO-X to detect new white dwarf companions to AFGK stars. We present the first results of our survey with the extreme AO instrument MagAO-X, called the Pup Search, of 18 targets with seven new candidate companions, five of which are confirmed to be white dwarfs. We discuss the new detections in the context of previous surveys and other detection metric sensitivities and show that we are sensitive to a region not probed by other surveys. Finally we discuss the future of the Pup Search in light of developing technologies.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

493  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

8  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

0  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
